# Usage

Run setup cell and cell below "Chapter 4"-heading to interactively simulate different schemes for the linear advection equation as well as the SPS. 

# Setup

In [1]:
%matplotlib inline

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import src.config as config 
import src.animation as animation 
import matplotlib.pyplot as plt 

import numpy as np

# Chapter 4

## The linear advection equation

Note that the option limiter is only valid for the second-order upwind scheme (SOU) with limiter. 


In [2]:

import src.advection_schemes as advection_schemes
import src.advection_tests as advection_tests

algorithms = {
    "centered difference":advection_schemes.CenteredDifferenceScheme,
    "upwind": advection_schemes.UpwindScheme,
    "second-order upwind": advection_schemes.SecondOrderUpwindScheme,
    "sou with limiter": advection_schemes.SOLimiterScheme,
    "lax-wendroff": advection_schemes.LaxWendroffScheme
}

tests = {
    "gaussian": advection_tests.gaussian1D,
    "tophat": advection_tests.tophat1D
}


def getBaseConfig():
    c = config.generateConfig(dt=1e-4, t0=0)
    c["dt"] = 1e-3
    c["domainSize"] = 5
    c["xlim"] =  [0, 5]
    c["densityYlim"] = [0, 1.6]
    c["resolution"] = 256
    c["timeOrder"] = 1
    c["stencilOrder"] = 2
    c["dimension"] = 1
    c["debug"] = False
    c["slowDown"] = 1
    c["tEnd"] = 1
    c["gravity"] = 1
    c["outputTimestep"] = False
    c["alpha"] = 0.1
    c["useAdaptiveTimestep"] = True
    c["usePeriodicBC"] = False
    c["dpi"] = 200
    c["size"] = (3.54 * 2, 3.54)#(3.54 * 1.5, 3.54)
    c["savePlots"] = True
    c["cfl"] = 0.1
    return c

def f(t, resolution, time_order, scheme, test, limiter, take_snapshot):
    c = getBaseConfig()
    c["resolution"] = resolution
    c["timeOrder"] = time_order
    c["tEnd"] = t
    c["savePlots"] = take_snapshot
    c["fluxLimiter"] = limiter
    if take_snapshot:
        c["dpi"] = 600
        c["size"] = (3.54 * 2, 3.54)
    else:
        c["dpi"] = 120
        c["size"] = (3.54 * 2, 3.54)

    solver = algorithms[scheme](c, tests[test])
    solver.run()
    animation.drawPrettyFrame(solver, advection=True)
    plt.show()

interactive_plot = interactive(f, 
t=widgets.FloatSlider(min=0, max=3, step=0.1, value=1.5), 
resolution=widgets.IntSlider(min=16, max=256, step=4, value=128), 
time_order= widgets.IntSlider(min=1, max=4, step=1, value=1),
scheme = ["centered difference", "upwind", "second-order upwind", "sou with limiter", "lax-wendroff"],
test = ["gaussian", "tophat"],
limiter = [
    "DONORCELL"  , 
    "BEAMWARMING",
    "LAXWENDROFF",
    "MINMOD"     ,
    "VANALBADA"  ,   
    "SUPERBEE"   ,
    "MC"         ,
    "VANLEER"
],
take_snapshot = False
)
output = interactive_plot.children[-1]
output.layout.height = '500px'
interactive_plot

interactive(children=(FloatSlider(value=1.5, description='t', max=3.0), IntSlider(value=128, description='reso…

## The phase scheme

In [5]:
import src.wave_schemes as wave_schemes
import src.phase_schemes as phase_schemes
import src.fluid_schemes as fluid_schemes
import src.tests as tests 


def getBaseConfig():
    c = config.generateConfig(dt=1e-4, t0=0)
    c["dt"]                  = 1e-4
    c["domainSize"]          = 1
    c["xlim"]                = [0, 1]
    c["densityYlim"]         = [0, 2]
    c["resolution"]          = 128
    c["timeOrder"]           = 1
    c["dimension"]           = 1
    c["debug"]               = False
    c["slowDown"]            = 1
    c["tEnd"]                = 1
    c["outputTimestep"]      = False
    c["useAdaptiveTimestep"] = True
    c["usePeriodicBC"]       = False
    c["gravity"]             = 0
    c["nThreads"]            = 4
    c["fps"]                 = 20
    return c


def ftcsConfig(c):
    c["stencilOrder"] = 6
    c["timeOrder"]    = 2

def cnConfig(c):
    c["stencilOrder"] = 1
    c["timeOrder"]    = 1

def spectralConfig(c):
    c["usePeriodicBC"] = True


def upwindConfig(c):
    c["stencilOrder"] = 1
    c["timeOrder"]    = 4

def upwindWithFrictionConfig(c):
    c["stencilOrder"] = 1
    c["timeOrder"]    = 2
    c["friction"] = 0.2

def upwindWithoutDiffusionConfig(c):
    c["stencilOrder"] = 1
    c["timeOrder"]    = 4
    c["turnOffDiffusion"] = True

def upwindWithoutConvectionConfig(c):
    c["stencilOrder"] = 1
    c["timeOrder"]    = 4
    c["turnOffConvection"] = True 

def hoUpwindConfig(c):
    c["stencilOrder"] = 4
    c["timeOrder"] = 2

def hoUpwindWithoutDiffusionConfig(c):
    c["stencilOrder"] = 4
    c["timeOrder"] = 2
    c["turnOffDiffusion"] = True

def hoUpwindWithoutConvectionConfig(c):
    c["stencilOrder"] = 4
    c["timeOrder"] = 2
    c["turnOffConvection"] = True 

def lwUpwindConfig(c):
    c["stencilOrder"] = 2
    c["timeOrder"] = 1

def ftcsConvectiveConfig(c):
    c["stencilOrder"] = 4
    c["timeOrder"] = 2

def ftcsConvectiveWithoutSourceConfig(c):
    c["stencilOrder"] = 4
    c["timeOrder"] = 2
    c["turnOffSource"] = True

def ftcsConvectiveWithoutConvectionConfig(c):
    c["stencilOrder"] = 4
    c["timeOrder"] = 2
    c["turnOffConvection"] = True 

def ftcsConvectiveWithoutDiffusionConfig(c):
    c["stencilOrder"] = 4
    c["timeOrder"] = 2
    c["turnOffDiffusion"] = True

def musclHancockConfig(c):
    c["stencilOrder"] = 2
    c["timeOrder"] = 2

def standingWaveConfig(c):
    c["tEnd"] = 6
    c["slowDown"] = 1
    c["resolution"] = 64

def oscillatorEigenstateConfig(c):
    c["domainSize"] = 6
    c["xlim"] = [0, 6]
    c["densityYlim"] = [0, 10]

def oscillatorCoherentStateConfig(c):
    c["domainSize"] = 14
    c["xlim"] = [0, 14]
    c["densityYlim"] = [0, 10]
    c["tEnd"] = 4 * np.pi

def fastOscillatorCoherentStateConfig(c):
    c["domainSize"] = 4
    c["xlim"] = [0, 4]
    c["densityYlim"] = [0, 10]
    c["tEnd"] = np.pi

def infiniteWellConfig(c):
    c["domainSize"] = 1
    c["xlim"] = [0, 1]
    c["densityYlim"] = [0, 4]
    c["usePeriodicBC"] = False

def perturbationWaveConfig(c):
    c["usePeriodicBC"] = True
    c["resolution"] = 64
    c["tEnd"] = 1/(2*np.pi*3*0.5)
    c["domainSize"] = 1
    c["xlim"] = [0, 1]
    c["densityYlim"] = [0.98, 1.02]
    c["phaseYlim"] = [-0.01, 0.01]
    c["slowDown"] = 5/c["tEnd"] 


def solitonConfig(c):
    c["usePeriodicBC"] = True
    c["resolution"] = 512
    c["tEnd"] = 10
    c["domainSize"] = 10
    c["xlim"] = [0, 10]
    c["densityYlim"] = [0, 10]
    c["slowDown"] = 10
    c["gravity"] = 1

def expandingSolitonConfig(c):
    c["usePeriodicBC"] = True
    c["resolution"] = 512
    c["tEnd"] = 10
    c["domainSize"] = 10
    c["xlim"] = [0, 10]
    c["densityYlim"] = [0, 10]
    c["slowDown"] = 10
    c["gravity"] = 1
    c["useCosmology"] = True

def li1Config(c):
    c["resolution"] = 128
    c["tEnd"] = .25
    c["domainSize"] = 4
    c["xlim"] = [0, 4]
    c["densityYlim"] = [0, 25]
    c["slowDown"] = 20

def li2Config(c):
    c["usePeriodicBC"] = True
    c["resolution"] = 4096
    c["t0"] = 0.0025
    c["tEnd"] = 0.01
    c["domainSize"] = 10
    c["xlim"] = [-0.3, 2.3]
    c["densityYlim"] = [4.5, 6.5]
    c["phaseYlim"] = [-0.05, 0.05]
    c["slowDown"] = 500

def li3Config(c):
    c["usePeriodicBC"] = True
    c["resolution"] = 512
    c["tEnd"] = .005
    c["domainSize"] = 1
    c["xlim"] = [0, 1]
    c["densityYlim"] = [0, 16]
    c["slowDown"] = 2000


def travellingWavePacketConfig(c):
    c["usePeriodicBC"] = False
    c["resolution"] = 512
    c["tEnd"] = .005
    c["domainSize"] = 1
    c["xlim"] = [0, 1]
    c["densityYlim"] = [0, 16]
    c["slowDown"] = 2000


def perturbationWave2DConfig(c):
    c["dimension"] = 2
    c["usePeriodicBC"] = True
    c["resolution"] = 16
    c["tEnd"] = 1/(2*np.pi*3*0.5)
    c["domainSize"] = 1
    c["xlim"] = [0, 1]
    c["densityYlim"] = [0.6, 1.4]
    c["cfl"] = .2
    c["slowDown"] = 2/c["tEnd"] 

test_list = {
    "standing wave": [tests.standingWave, standingWaveConfig, None],
    "harmonic oscillator convergence": [tests.generate1DUniform, oscillatorEigenstateConfig, lambda x: tests.oscillatorPotential1D(x, x0 = 3)],
    "harmonic oscillator eigenstate": [tests.oscillatorEigenstate1D, oscillatorEigenstateConfig, lambda x: tests.oscillatorPotential1D(x, x0 = 3)],
    "harmonic oscillator coherent state": [tests.oscillatorCoherentState1D, oscillatorCoherentStateConfig, lambda x: tests.oscillatorPotential1D(x, x0 = 7)],
    "infinite well": [tests.infiniteWell1D, infiniteWellConfig, None],
    "standing wave packet": [lambda x, dx, t: tests.li1(x, dx, t, x0=2), li1Config, None],
    "quasi-shock": [lambda x, dx, t: tests.li2(x, dx, t, x0 =5), li2Config, None],
    "wave packet collision": [tests.li3, li3Config, None],
    "travelling wave packet": [tests.travellingWavePacket, travellingWavePacketConfig, None],
    "perturbation wave": [tests.cosmological1D, perturbationWaveConfig, None],
    "soliton": [lambda xx, dx, t: tests.cosmological1D(xx, dx, t, eps=5e-3, Lx=10, N=10), solitonConfig, None],
    "expanding_soliton": [lambda xx, dx, t: tests.cosmological1D(xx, dx, t, eps=5e-3, Lx=10, N=10), expandingSolitonConfig, None],
    "perturbation wave 2D": [lambda x, y, dx, t: tests.cosmological2D(x, y, dx, t, Lx = 1, Ly = 1, N = 3, eps= 0.1), perturbationWave2DConfig, None]
}

scheme_list = {
    "wave - ftcs (forward in time, centered in space)": [wave_schemes.FTCSScheme, ftcsConfig],
    "wave - crank-nicolson": [wave_schemes.CNScheme, cnConfig],
    "wave - spectral": [wave_schemes.SpectralScheme, spectralConfig],
    "phase - upwind": [phase_schemes.UpwindScheme, upwindConfig],
    "phase - upwind with friction": [phase_schemes.UpwindScheme, upwindWithFrictionConfig],
    "phase - upwind without quantum pressure": [phase_schemes.UpwindScheme, upwindWithoutDiffusionConfig],
    "phase - upwind without convection": [phase_schemes.UpwindScheme, upwindWithoutConvectionConfig],
    "phase - ho-upwind": [phase_schemes.HOUpwindScheme, hoUpwindConfig],
    "phase - ho-upwind without diffusion": [phase_schemes.HOUpwindScheme, hoUpwindWithoutDiffusionConfig],
    "phase - ho-upwind without convection": [phase_schemes.HOUpwindScheme, hoUpwindWithoutConvectionConfig],
    "phase - lw-upwind": [phase_schemes.LaxWendroffUpwindScheme, lwUpwindConfig],
    "phase - ftcs-convective": [phase_schemes.FTCSConvectiveScheme, ftcsConvectiveConfig],
    "phase - ftcs-convective without diffusion": [phase_schemes.FTCSConvectiveScheme, ftcsConvectiveWithoutDiffusionConfig],
    "phase - ftcs-convective without convection": [phase_schemes.FTCSConvectiveScheme, ftcsConvectiveWithoutDiffusionConfig],
    "phase - ftcs-convective without source": [phase_schemes.FTCSConvectiveScheme, ftcsConvectiveWithoutSourceConfig],
    "fluid - muscl-hancock": [fluid_schemes.MUSCLHancock, musclHancockConfig],
}

def pick_scheme(scheme_name, test_name):
    c = getBaseConfig()
    test, testConfig, potential = test_list[test_name]
    scheme, schemeConfig = scheme_list[scheme_name]
    schemeConfig(c)
    testConfig(c)
    
    c["dpi"] = 120
    c["size"] = (3.54 * 2, 3.54)

    solver = scheme(c, test)
    solver.setExternalPotentialFunction(potential)
    return solver 


scheme_picker = interactive(pick_scheme, 
scheme_name = scheme_list.keys(),
test_name = test_list.keys(),
)
scheme_picker

interactive(children=(Dropdown(description='scheme_name', options=('wave - ftcs (forward in time, centered in …

In [6]:

def run(t, take_snapshot, back_in_time):
    solver = scheme_picker.result
    solver.tEnd = solver.config["tEnd"] * t/100
    solver.config["savePlots"] = take_snapshot

    if take_snapshot:
        solver.config["dpi"] = 600
        solver.config["size"] = (3.54 * 2, 3.54)
    else:
        solver.config["dpi"] = 120
        solver.config["size"] = (3.54 * 2, 3.54)

    solver.run(enableBackward=back_in_time)
    animation.drawPrettyFrame(solver)
    plt.show()


interactive_plot= interactive(run,
t=widgets.FloatSlider(min=0, max=1000, step=1, value=0), 
take_snapshot = False,
back_in_time = False
)

output = interactive_plot.children[-1]
output.layout.height = '500px'
interactive_plot

interactive(children=(FloatSlider(value=0.0, description='t', max=1000.0, step=1.0), Checkbox(value=False, des…